In [ ]:
from google import genai
from google.genai.types import GenerateContentConfig,Tool
from google.colab import userdata
import json
import subprocess

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)
config_20 = GenerateContentConfig(
    temperature=0.1,
    top_p=1,system_instruction="You are a Relationship Manager with excellent financial analysis knowledge at an esteemed US bank.You are proficient at market reasearch for fininacial analysis.",
    tools=[Tool(googleSearch={})]
)
prompt = '''1.Search for recent news on C-suite changes, including CEO appointments, executive leadership transitions, and new hires for key positions like CFO, COO, or CTO at American Crafts L.C.
            2.First search any recent leadership change news on American Crafts L.C. official website.
            3.Focus on articles that mention leadership shifts, restructuring of the executive team, or the appointment of new top executives at American Crafts L.C.
            3.Ensure the results are from trusted sources like Bloomberg, Reuters, Business Insider, The Wall Street Journal, Financial Times and similar.
            4.Filter results to only include articles from the past 6 months to 1 year
            5.Exclude duplicate or broken links

            **Criteria for Results**:
- Provide a **one-line summary** for each relevant leadership new item with the **latest corresponding URL**.
- Ensure no duplication of news, whether from the same or different sources.
- Prioritize results with high confidence scores and rank results in **descending order** by:
  - **Latest date** (most recent news first).
  - **Relevance** to American Crafts L.C. leadership changes (focus on leadership changes, executive leadership update etc).
  - **Credibility of the source** (give preference to more trusted sources like American Crafts L.C.'s official website, Bloomberg, Reuters, etc.).
- Focus on web pages containing keywords like: **leadership overhaul, leadership shifts, c-suite change, New CEO/CFO/COO/CTO, executive leadership update, board of directors change, corporate leadership news, new top executive, company announces new leadership, executive team restructuring**.etc
- **Exclude broken and blocked and invalid URL's**'''


prompt1 = '''Find news on recent C-suite changes (CEO, CFO, COO, CTO) including executive appointments at American Crafts L.C. from it's official website including trusted sources (e.g., Bloomberg, Reuters, Business Insider, Financial Times, The Verge, WSJ and similar)
             within the last 6 months to 1 year. Give me a reported date along with **one-line summary** and the associated URL.Exclude duplicate and broken links. Keep your search confined to leadership cahnges at American Crafts L.C. and no variations of company name thereof.
             If you don't find any news then give the output as "No news found"'''

prompt2 = '''1.Search for news regarding C-suite changes (CEO, CFO, COO, CTO) and executive appointments at Sony Corporation from Sony Corporation's official website and other trusted sources like Bloomberg, Reuters, Business Insider, Financial Times, and WSJ and similar within the last 6 months to 1 year.
             2.Focus on web pages containing keywords like: **leadership overhaul, leadership shifts, c-suite change, New CEO/CFO/COO/CTO, executive leadership update, board of directors change, corporate leadership news, new top executive, company announces new leadership, executive team restructuring**.etc
             3.Give me a reported date along with **one-line summary** and the associated URL.
             4.Exclude duplicate and broken links.
          '''

prompt3 = '''1.Search for recent news on C-suite changes at American Crafts L.C., including CEO appointments, executive leadership transitions, and new hires for key positions like CFO, COO, or CTO.
             2.Begin by checking American Crafts L.C.'s official website for any recent leadership change announcements or news.
             3.Focus on articles that specifically mention leadership shifts, restructuring of the executive team, or the appointment of new top executives at American Crafts L.C..
             4.Ensure results come from trusted sources like Bloomberg, Reuters, Business Insider, The Wall Street Journal, Financial Times, and similar.
             5.Filter results to only include articles from the past 6 months to 1 year, ensuring that the information is up-to-date.
             6.Exclude duplicate, broken, blocked, or invalid URLs.

             **Criteria for Results**:
- Provide a **one-line summary** for each relevant leadership change with the **latest corresponding URL**.
- **Avoid duplication** of news, whether from the same or different sources.
- Prioritize results with high confidence scores and rank results in **descending order** by:
  - **Latest date** (most recent news first).
  - **Relevance** to American Crafts L.C.'s leadership changes (focus on leadership changes, executive updates etc).
  - **Credibility of the source** (prefer trusted sources like American Crafts L.C.'s official website, Bloomberg, Reuters, etc.).
- Focus on web pages containing keywords like: **leadership overhaul, leadership shifts, c-suite change, New CEO/CFO/COO/CTO, executive leadership update, board of directors change, corporate leadership news, new top executive, company announces new leadership, executive team restructuring**.etc
- **Exclude broken and blocked and invalid URL's**'''

# prompt1 = '''1. Please note that you are a renowned financial market researcher.
# 2. Provide me with the latest news, from today to the past year, on Sony Corporation's M&A activities from verified sources.
# 3. Focus on keywords such as "Sony Corp's Mergers and Acquisitions," "Sony Corp's M&A," "Sony Corp's Acquisitions," "Sony Corp's Divestitures," and "Sony Corp's Takeovers."
# 4. Search sources like Sony's Investor Relations Website, major financial news outlets (Bloomberg, Reuters, Wall Street Journal, Financial Times, CNBC), and dedicated M&A news sources (Mergermarket, DealBook, Pitchbook, The Deal), which offer in-depth coverage of mergers and acquisitions. Also, check the details of mergers and acquisitions in Sony's filings with the U.S. Securities and Exchange Commission (SEC).
# 5. Provide me with complete sentences, not just the title. Rank the links by their reliability, latest date, and relevance to the search query.
# 6. Exclude any repeated news from the same or different sources in the output.  '''

# prompt2 = '''
# 1.Search for F.W.Webb's M&A activities in the last 2 year from reliable and trusted resources such as:

# 1.1. F.W.Webb's Investor Relations website or check F.W. Webb's official website directly for investor relations information.
# 1.2. Major financial news outlets: Bloomberg, Reuters, Wall Street Journal, Financial Times, CNBC, The Verge
# 1.3. Dedicated M&A news sources: Mergermarket, DealBook, Pitchbook, The Deal
# 1.4. F.W.Webb's filings with the U.S. SEC
# 1.5  Not confining to these sources search other credible sources from major financial publications and financial journals.

# 2. Provide a one-line summary for each relevant M&A news item with the corresponding latest uri.
# 3. Exclude any duplicate news from the same or different sources.
# 4. Rank the output by:
# - Latest date
# - Relevance to F.W.Webb's M&A activities
# - Credibility of the source (F.W.Webb's official site, Bloomberg, Reuters, etc.)
# 5. Ensure that the uri's provided by you are working and from trusted sources only.
# '''

# prompt3 = '''Search for American Crafts's M&A activities in the last year, focusing on the following:

# 1. American Crafts's official website (Investor Relations section).
# 2. Trusted financial news sources like Bloomberg, Reuters, Wall Street Journal, Financial Times, CNBC, The Verge.
# 3. Dedicated M&A sources such as Mergermarket, DealBook, Pitchbook, The Deal.
# 4. Details of mergers and acquisitions in Sony's filings with the U.S. SEC (Securities and Exchange Commission).

# **Criteria for Results**:
# - Provide a **one-line summary** for each relevant M&A activity and include the corresponding uri
# - Ensure no duplication of information from the same or different sources.
# - Rank results in **descending order** by:
#   - **Latest date**
#   - **Relevance to American Crafts's M&A activities**
#   - **Credibility of the source** (with priority given to more trusted sources like American Crafts's official website, Bloomberg, Reuters, etc.)
# - Ensure all links are **working** and from trusted sources only.
# - Focus on web pages containing keywords like: **partnership, takeover, acquires, acquired by, merged**, etc.
# - **Exclude broken links** and prioritize high-confidence sources for accuracy and reliability.

# Start by searching American Crafts L.C. official website first, followed by major financial journals and M&A-specific sources.
# '''

# prompt4 = '''Search for American Crafts L.C. M&A activities over the last two years using the following trusted sources:

# 1. American Crafts L.C. official website or its Investor Relations section.
# 2. Major financial news outlets like Bloomberg, Reuters, Wall Street Journal, Financial Times, CNBC, and The Verge.
# 3. Dedicated M&A sources such as Mergermarket, DealBook, Pitchbook, The Deal.
# 4. American Crafts L.C. filings with the U.S. SEC (Securities and Exchange Commission).
# 5. Other reliable financial sources from well-known publications and journals.

# **Criteria for Results**:
# - Provide a **one-line summary** for each relevant M&A news item with the **latest corresponding URL**.
# - Ensure no duplication of news, whether from the same or different sources.
# - Prioritize results with high confidence scores and rank results in **descending order** by:
#   - **Latest date** (most recent news first).
#   - **Relevance** to American Crafts L.C. M&A activities (focus on acquisitions, mergers, or related transactions).
#   - **Credibility of the source** (give preference to more trusted sources like American Crafts L.C.'s official website, Bloomberg, Reuters, etc.).
# - Focus on web pages containing keywords like: **partnership, takeover, acquires, acquired by, merged**, etc.
# - **Exclude broken and blocked and invalid URL's**
# '''
resp_20 = client.models.generate_content(model = "gemini-2.0-flash",config=config_20,contents=prompt1)
print(resp_20.text)
#print(resp_20.candidates[0].grounding_metadata.search_entry_point.rendered_content)
#print(resp_20.candidates[0].grounding_metadata.grounding_chunks)
#print(resp_20.candidates[0].grounding_metadata)

# grounding_metadata = resp_20.candidates[0].grounding_metadata.grounding_chunks
# grounding_metadata_json = json.dumps(grounding_metadata, indent=4)
# result = subprocess.run(['jq', '.', '<(echo "' + grounding_metadata_json + '")'], capture_output=True, text=True, shell=True)
# print(result.stdout)

# # Write the grounding metadata to a file
# with open('grounding_metadata.json', 'w') as f:
#     f.write(grounding_metadata_json)

# Use jq to process the JSON file
#subprocess.run(["jq", ".", "grounding_metadata.json"])

#genai.configure(api_key=GOOGLE_API_KEY)

My search did not yield any relevant news articles about C-suite changes at American Crafts L.C. within the last 6 months to 1 year.

Therefore, my output is: "No news found"



In [ ]:
resp_20.candidates[0].grounding_metadata.grounding_chunks

[GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='lg.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYtXsm5nh4gPZS4eNgTsm8765XVkwTv4xZtYDHz-_Z-N4vDTaE-cVLPVoNfP6_XIZBV-6Lo_Y8UY7uP8efgEdSWyZYQxo7ad9NFcQU2gz9-KuzSCq6SoSDL-bfnCPCOn')),
 GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='google.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYscsnDbmbxrCB2XkPXrGHU3L2fCLsZ6wteyEjF6IvA20cbQD0VzGIp1CDCaP13dbgMoqZ5XxUcxSvGAqxF3nn5Kbt4qoeAm34kq-NQMgAXfz5lChJ5IHKKLzZnDT6MP9oMUMXdSX6NnIeASf1c63JVwE0xE'))]

In [ ]:
uris = [chunk.web.uri for chunk in resp_20.candidates[0].grounding_metadata.grounding_chunks]
l = resp_20.text

In [ ]:
uris

['https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYtXsm5nh4gPZS4eNgTsm8765XVkwTv4xZtYDHz-_Z-N4vDTaE-cVLPVoNfP6_XIZBV-6Lo_Y8UY7uP8efgEdSWyZYQxo7ad9NFcQU2gz9-KuzSCq6SoSDL-bfnCPCOn',
 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYscsnDbmbxrCB2XkPXrGHU3L2fCLsZ6wteyEjF6IvA20cbQD0VzGIp1CDCaP13dbgMoqZ5XxUcxSvGAqxF3nn5Kbt4qoeAm34kq-NQMgAXfz5lChJ5IHKKLzZnDT6MP9oMUMXdSX6NnIeASf1c63JVwE0xE']

In [ ]:
c_suite_link = {}
if (("No news found").lower() not in l.lower()):
  titles = [title.segment.text for title in resp_20.candidates[0].grounding_metadata.grounding_supports]
  summ_link = list(zip(titles,uris))
  c_suite_link = dict(summ_link)
c_suite_json = json.dumps(c_suite_link, indent=4)
with open('C_Suite_activity.json', 'w') as f:
  f.write(c_suite_json)